In [1]:
import pandas as pd
from pypinyin import lazy_pinyin as lpy, Style
from pywubi import wubi as wb
import jieba
from jieba import analyse
jieba.load_userdict(r'F:\SCU\大创\dataResource\All_Abbr.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jastaron\AppData\Local\Temp\jieba.cache
Loading model cost 0.478 seconds.
Prefix dict has been built successfully.


In [2]:
PyIniDict = {
    'b':'1', 'p':'2', 'm':'3', 'f':'4', 'd':'5', 't':'6', 'n':'7', 'l':'7',
    'g':'8', 'k':'9', 'h':'A', 'j':'B', 'q':'C', 'x':'D', 'zh':'E', 'ch':'F',
    'sh':'G', 'r':'H', 'z':'E', 'c':'F', 's':'G', 'y':'I', 'w':'J', '':'I'
}
PyFinalDict = {
    'a':'1', 'o':'2', 'e':'3', 'i':'4', 'u':'5', 'v':'5', 'yu':'5', 'ai':'6',
    'ei':'7', 'ui':'8', 'ao':'9', 'ou':'A', 'iu':'B', 'ie':'C', 'ye':'C',
    've':'D', 'yue':'D', 'ue':'D', 'er':'E', 'an':'F', 'en':'G', 'in':'H',
    'un':'I', 'ven':'J', 'yun':'J', 'ang':'F', 'eng':'I', 'ing':'H',
    'ong':'K', 'yin':'H','you':'A', 'uang':'L', 'iang':'M', 'iong':'N',
    'ua':'O', 'wen':'I', 'yan':'P', 'ian':'P', 'yang':'M', 'yong':'N',
    'uo':'Q', 'wo':'Q', 'ia':'R', 'ya':'R', 'uan':'S', 'wan':'S', 'wa':'O',
    'iao':'T','yao':'T'
}
strokesDict = {
    0:'0',1:'1',2:'2',3:'3',4:'4',5:'5',6:'6',7:'7',8:'8',9:'9',
    10:'a',11:'b',12:'c',13:'d',14:'e',15:'f',16:'g',17:'h',18:'i',19:'j',
    20:'k',21:'l',22:'m',23:'n',24:'o',25:'p',26:'q',27:'r',28:'s',29:'t',
    30:'u',31:'v',32:'w',33:'x',34:'y',35:'z',36:'A',37:'B',38:'C',39:'D',
    40:'E',41:'F',42:'G',43:'H',44:'I',45:'J',46:'K',47:'L',48:'M',49:'N',
    50:'O',51:'P',52:'Q',53:'R',54:'S',55:'T',56:'U',57:'V',58:'W',59:'X',
    60:'Y',61:'Z'
}
city_error = {
    '达县':'达州市',
    '渡口':'攀枝花市'
}

In [3]:
def fin(strokes_path):
    # 如果返回 0, 则也是在unicode中不存在kTotalStrokes字段
    strokes = []
    with open(strokes_path, 'r') as fr:
        for line in fr:
            strokes.append(int(line.strip()))
    return strokes
def get_stroke(c,strokes):
    unicode_ = ord(c)
    if 13312 <= unicode_ <= 64045:
        return strokes[unicode_ - 13312]
    elif 131072 <= unicode_ <= 194998:
        return strokes[unicode_ - 80338]
    else:
        print("c should be a CJK char, or not have stroke in unihan data.")
        # can also return 0

In [4]:
total_locations = pd.read_csv(r'F:\SCU\大创\dataResource\SC_CQ_total_load.csv')
build_dict_locations = pd.read_csv(r'F:\SCU\大创\dataResource\SC_CQ_total_history_load.csv')
hill_image = pd.read_csv(r'F:\SCU\大创\dataResource\hill_image_1.csv')
strokes = fin(r'F:\SCU\大创\python导入\strokes.txt')
totalDict = {}

In [5]:
layerDict = {'四川省': {}}
for i in range(len(build_dict_locations)):
    PAC = build_dict_locations.iloc[i][0]
    province = build_dict_locations.iloc[i][1]
    city = build_dict_locations.iloc[i][2]
    county = build_dict_locations.iloc[i][3]
    if city not in layerDict[province]:
        layerDict[province][city] = {}
    layerDict[province][city][county] = PAC

county_total = build_dict_locations['county'].tolist()
PAC_total = build_dict_locations['PAC'].tolist()
county_dict = dict(zip(county_total,PAC_total))

In [6]:
layerDict

{'四川省': {'成都市': {'锦江区': 510104,
   '东城区': 510104,
   '青羊区': 510105,
   '西城区': 510105,
   '金牛区': 510106,
   '武侯区': 510107,
   '成华区': 510108,
   '龙泉驿区': 510112,
   '青白江区': 510113,
   '新都区': 510114,
   '温江区': 510115,
   '双流区': 510116,
   '郫都区': 510117,
   '郫县': 510117,
   '新津区': 510118,
   '金堂县': 510121,
   '大邑县': 510129,
   '蒲江县': 510131,
   '都江堰市': 510181,
   '灌县': 510181,
   '彭州市': 510182,
   '彭县': 510182,
   '邛崃市': 510183,
   '崇州市': 510184,
   '崇庆': 510184,
   '简阳市': 510185},
  '自贡市': {'自流井区': 510302,
   '贡井区': 510303,
   '大安区': 510304,
   '沿滩区': 510311,
   '荣县': 510321,
   '富顺县': 510322},
  '攀枝花市': {'东区': 510402,
   '西区': 510403,
   '仁和区': 510411,
   '米易县': 510421,
   '盐边县': 510422},
  '泸州市': {'江阳区': 510502,
   '纳溪区': 510503,
   '龙马潭区': 510504,
   '泸县': 510521,
   '合江县': 510522,
   '叙永县': 510524,
   '古蔺县': 510525},
  '德阳市': {'旌阳区': 510603,
   '罗江区': 510604,
   '中江县': 510623,
   '广汉市': 510681,
   '什邡市': 510682,
   '绵竹市': 510683},
  '绵阳市': {'涪城区': 510703,
   '游仙区': 510704,
   '安州区': 51

In [7]:
county_dict

{'锦江区': 510104,
 '东城区': 510104,
 '青羊区': 510105,
 '西城区': 510105,
 '金牛区': 510106,
 '武侯区': 510107,
 '成华区': 510108,
 '龙泉驿区': 510112,
 '青白江区': 510113,
 '新都区': 510114,
 '温江区': 510115,
 '双流区': 510116,
 '郫都区': 510117,
 '郫县': 510117,
 '新津区': 510118,
 '金堂县': 510121,
 '大邑县': 510129,
 '蒲江县': 510131,
 '都江堰市': 510181,
 '灌县': 510181,
 '彭州市': 510182,
 '彭县': 510182,
 '邛崃市': 510183,
 '崇州市': 510184,
 '崇庆': 510184,
 '简阳市': 510185,
 '自流井区': 510302,
 '贡井区': 510303,
 '大安区': 510304,
 '沿滩区': 510311,
 '荣县': 510321,
 '富顺县': 510322,
 '东区': 510402,
 '西区': 510403,
 '仁和区': 510411,
 '米易县': 510421,
 '盐边县': 510422,
 '江阳区': 510502,
 '纳溪区': 510503,
 '龙马潭区': 510504,
 '泸县': 510521,
 '合江县': 510522,
 '叙永县': 510524,
 '古蔺县': 510525,
 '旌阳区': 510603,
 '罗江区': 510604,
 '中江县': 510623,
 '广汉市': 510681,
 '什邡市': 510682,
 '绵竹市': 510683,
 '涪城区': 510703,
 '游仙区': 510704,
 '安州区': 510705,
 '安县': 510705,
 '三台县': 510722,
 '盐亭县': 510723,
 '梓潼县': 510725,
 '北川羌族自治县': 510726,
 '平武县': 510727,
 '江油市': 510781,
 '利州区': 510802,
 '昭化区': 510811,
 '朝天区': 

In [8]:
def words_abbreviation(locations):
    nation_keywords = ['自治','土家族','苗族','藏族','彝族','羌族']
    locations_keywords = ['省','市','州','区','县']
    for i in range(len(locations)):
        radix = 0
        for n_kw in nation_keywords:
            if n_kw in locations[i]:
                locations[i] = locations[i].split(n_kw)[0]
        while any(k in locations[i] for k in locations_keywords) and len(locations[i]) > 2:
            locations[i] = locations[i].split(locations_keywords[radix])[0]
            radix += 1

In [9]:
countyAbbr = county_total.copy()
words_abbreviation(countyAbbr)
countyAbbr

['锦江',
 '东城',
 '青羊',
 '西城',
 '金牛',
 '武侯',
 '成华',
 '龙泉驿',
 '青白江',
 '新都',
 '温江',
 '双流',
 '郫都',
 '郫县',
 '新津',
 '金堂',
 '大邑',
 '蒲江',
 '都江堰',
 '灌县',
 '彭州',
 '彭县',
 '邛崃',
 '崇州',
 '崇庆',
 '简阳',
 '自流井',
 '贡井',
 '大安',
 '沿滩',
 '荣县',
 '富顺',
 '东区',
 '西区',
 '仁和',
 '米易',
 '盐边',
 '江阳',
 '纳溪',
 '龙马潭',
 '泸县',
 '合江',
 '叙永',
 '古蔺',
 '旌阳',
 '罗江',
 '中江',
 '广汉',
 '什邡',
 '绵竹',
 '涪城',
 '游仙',
 '安',
 '安县',
 '三台',
 '盐亭',
 '梓潼',
 '北川',
 '平武',
 '江油',
 '利',
 '昭化',
 '朝天',
 '旺苍',
 '青川',
 '剑阁',
 '苍溪',
 '船山',
 '安居',
 '蓬溪',
 '大英',
 '射洪',
 '',
 '东兴',
 '威远',
 '资中',
 '隆昌',
 '',
 '沙湾',
 '五通桥',
 '金口河',
 '金口河工农',
 '犍为',
 '井研',
 '夹江',
 '沐川',
 '峨边',
 '马边',
 '峨眉山',
 '顺庆',
 '高坪',
 '嘉陵',
 '南部',
 '营山',
 '蓬安',
 '仪陇',
 '西充',
 '阆中',
 '东坡',
 '彭山',
 '仁寿',
 '洪雅',
 '丹棱',
 '青神',
 '翠屏',
 '南溪',
 '叙',
 '江安',
 '长宁',
 '高县',
 '珙县',
 '筠连',
 '兴文',
 '屏山',
 '广安',
 '前锋',
 '岳池',
 '武胜',
 '邻水',
 '华蓥',
 '华云',
 '通川',
 '达川',
 '宣汉',
 '开江',
 '大竹',
 '渠县',
 '万源',
 '白沙工农',
 '雨城',
 '名山',
 '荥经',
 '汉源',
 '石棉',
 '天全',
 '芦山',
 '宝兴',
 '巴',
 '恩阳',
 '通江',
 '南江',
 '平昌',
 

In [10]:
city_total = build_dict_locations['city'].tolist()
l = len(city_total)
cityAbbr = []
for i in range(l):
    if city_total[i] not in cityAbbr:
        cityAbbr.append(city_total[i])
words_abbreviation(cityAbbr)
cityAbbr

['成都',
 '自贡',
 '攀枝花',
 '泸州',
 '德阳',
 '绵阳',
 '广元',
 '遂宁',
 '内江',
 '乐山',
 '南充',
 '眉山',
 '宜宾',
 '广安',
 '达州',
 '雅安',
 '巴中',
 '资阳',
 '阿坝',
 '甘孜',
 '凉山',
 '重庆']

In [11]:
hill_name = hill_image['NAME']
hill_PAC = hill_image['PAC']
hill_dict = dict(zip(hill_name,hill_PAC))
hill_dict

{'垭口梁': 511781,
 '乌云头': 500229,
 '卢家梁': 511781,
 '春天坡': 511781,
 '干龙洞坡': 511781,
 '观山': 511781,
 '百亩大梁': 511781,
 '獠牙寨': 511781,
 '姜池梁': 511781,
 '五个包': 500229,
 '粪桶梁': 511781,
 '老鸹山': 500229,
 '石人寨': 511781,
 '鱼泉山': 511781,
 '瓦岗寨': 500229,
 '木王山': 511781,
 '歪头山': 511781,
 '坟包梁': 511781,
 '麻里山': 511781,
 '川银山': 511921,
 '石梁子': 511781,
 '马棕寨': 511781,
 '严家寨': 511781,
 '城子山': 511921,
 '石板坡': 511902,
 '苦桃坡': 511781,
 '清坪寨': 511781,
 '两个田梁上': 511922,
 '三星寨': 511922,
 '刘家岭': 511921,
 '寨包梁': 511922,
 '马安寨': 511921,
 '大长梁': 511922,
 '六匹山': 511921,
 '王家寨': 511922,
 '红包梁': 511921,
 '龙池子': 511921,
 '梁口上': 511921,
 '拗盘寨': 511921,
 '雨台梁': 510821,
 '屋基梁': 511921,
 '小寨包': 511922,
 '观树梁': 511922,
 '贾家梁': 511921,
 '洪洞坡': 511781,
 '朱家梁': 511921,
 '黄裔观': 511922,
 '大操场梁上': 511921,
 '背阴\ue153': 510821,
 '刘家包': 511921,
 '汪家梁': 511922,
 '尖铧顶': 511921,
 '花椒园': 511781,
 '财神庙': 510821,
 '关门坡': 511922,
 '山坪梁': 511921,
 '退堂山': 511922,
 '木王坪': 511921,
 '云包树': 511921,
 '灯杆梁': 510821,
 '坟梁上': 511921,
 '天关田': 510821

In [66]:
hill_predict_index = []

In [77]:
def MatchCounty_new(locations_str):
    locations = list(jieba.cut(locations_str,cut_all=False,HMM=False))
    for i in range(len(locations),0,-1):
        index = i-1
        if len(locations[index]) < 2:
            locations.pop(index)

    match_group = []
    point = layerDict
    search_list = list(layerDict.keys())

    # ---------------------按层级查询---------------------
    words_abbreviation(locations)
    # print(locations)

    for l in locations:
        if point == layerDict['四川省']:
            city_bool = False
            for e in list(city_error.keys()):
                if l in e:
                    point = point[city_error[e]]
                    city_bool = True
                    break
            if city_bool:
                search_list = list(point.keys())
                continue
        for i in search_list:
            if l in i:
                match_group.append(i)
                point = point[i]
                break
        if not isinstance(point,dict):
            match_group.append(point)
            break
        search_list = list(point.keys())

    if len(match_group) == 4:
        return match_group


    # --------------------暴力检索县级---------------------
    # print('county')
    match_find = False
    county_names = list(county_dict.keys())
    for l in locations:
        for i in county_names:
            if l in i:
                match_PAC = county_dict[i]
                match_record = total_locations[total_locations['PAC']==match_PAC]
                match_find = True
                break
        if match_find:
            match_group = []
            for i in [1,2,3,0]:
                match_group.append(match_record.iloc[0][i])
            return match_group


    # ----------------------山峰检索------------------------
    # print('hills')
    # hill_predict_index.append(ind)
    hill_names = list(hill_dict.keys())
    for l in locations:
        for i in hill_names:
            if l in i:
                match_PAC = hill_dict[i]
                match_record = total_locations[total_locations['PAC']==match_PAC]
                match_find = True
                break
        if match_find:
            match_group = []
            for i in [1,2,3,0]:
                match_group.append(match_record.iloc[0][i])
            return match_group

    # print('none')
    return match_group

In [79]:
temp1 = 'asdfasdfasonasd你好但是我是不要咋说的发斯蒂芬沙坪坝'
location = MatchCounty_new(temp1)
print(location)

[nan, '重庆市', '沙坪坝区', 500106]


In [29]:
train = pd.read_csv(r'F:\SCU\大创\dataResource\Train.csv',keep_default_na=True)
train_PAC = train['PAC'].tolist()
train_str = train['str'].tolist()
train_str

['四川凤凰山王家涝',
 '四川万县地区中江石泉',
 '四川省万县双和乡',
 '四川万县地区忠县乌羊区石子乡',
 '四川万县地区巫山凤凰公社相柳池',
 '四川省涪陵地区涪陵酒店',
 '四川涪陵地区彭水条柘区大冈公社',
 '四川涪陵地区彭水桑柘区狮子洞',
 '四川涪陵彭水八区大垭乡',
 '四川涪陵地区丰都茶园区太平公社',
 '四川涪陵地区垫江县李家坡煤场',
 '四川涪陵（地区）石柱桥头区盐井公社二大队',
 '四川涪陵地区涪陵马武区兴隆公社大丛大队',
 '四川涪陵地区石柱三汇公社7大队药场',
 '四川涪陵地区涪陵新妙区两汇公社十字大队',
 '四川涪陵地区涪陵李渡区大柏公社山脚大队',
 '四川涪陵地区秀山溶溪水银厂附近',
 '四川涪陵（地区）彭水火石公社双龙大队',
 '四川涪陵地区黔江新建公社耕元大队',
 '四川涪陵地区武隆巷口凤山大队',
 '四川涪陵地区南川德隆公社白籽坝',
 '四川涪陵地区丰都茶园区武坪公社',
 '四川涪陵地区黔江县',
 '四川涪陵地区酉阳钟多区山阳公社秋谷大队',
 '四川涪陵地区武隆双河乡扁山',
 '四川涪陵地区酉阳大溪区大溪大队',
 '四川省渡口市市区山顶上',
 '四川重庆市南岸区老厂山边',
 '四川重庆市南岸区黄桷桠中药研究所',
 '四川北碚天生桥',
 '四川江津地区大足双河区多宝公社建角大队',
 '四川江津地区大足龙水玉龙公社五七中学',
 '四川省峨嵋县峨嵋山伏虎寺附近',
 '四川省峨眉县峨嵋山雷音寺',
 '四川江津地区江津西郊',
 '四川江津',
 '四川江津地区荣昌半边街',
 '四川江津地区荣昌双河区五.七煤矿附近',
 '四川江津合川合川盐井华川厂附近',
 '四川江津合川盐井华川厂附近',
 '四川合川宝顶山',
 '四川江津地区合川县盐井公社解放大队',
 '四川江津地区合川盐井铁厂附近',
 '四川江津地区永川石庙区九龙公社十大队',
 '四川江津地区永川通用机械厂',
 '四川江津（地区）永川石庙九龙公社一大队',
 '四川南川白雾坪',
 '四川省南川县三家车路旁山坡上',
 '四川涪陵地区南川三汇电厂沟',
 '四川南川县小河区头渡马咀岩',
 '四川南川县金佛山大河坝黄泥坡路边',
 '四川省南川县金佛山小河坝周家山',
 '四川涪陵南川永安公社',
 

In [69]:
test_predict = []
for i in range(len(train_str)):
    location = MatchCounty_new(train_str[i])
    if len(location) == 4:
        test_predict.append(location[3])
        continue
    test_predict.append(0)
test_predict

[511922,
 510623,
 0,
 500233,
 500237,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500102,
 500104,
 500108,
 500108,
 500109,
 500116,
 500116,
 500112,
 511181,
 500116,
 500116,
 500116,
 500116,
 500116,
 500116,
 500117,
 500116,
 500116,
 500116,
 500116,
 500116,
 500119,
 500119,
 500102,
 500119,
 500119,
 500119,
 500102,
 500119,
 500119,
 500119,
 500119,
 500119,
 500119,
 500119,
 500119,
 500119,
 500119,
 500119,
 500116,
 500116,
 500153,
 500155,
 500156,
 500102,
 500156,
 500229,
 500229,
 500229,
 500229,
 500229,
 500229,
 500229,
 500229,
 500229,
 500229,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500236,
 500237,
 500238,
 500238,
 500238,
 500238,
 500238,
 50023

In [76]:
from sklearn.metrics import classification_report
class_report = classification_report(train_PAC,test_predict)
print(class_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
      348301       0.00      0.00      0.00         1
      500101       0.00      0.00      0.00         4
      500102       0.60      1.00      0.75        21
      500104       0.25      1.00      0.40         1
      500108       1.00      1.00      1.00         2
      500109       1.00      1.00      1.00         1
      500111       0.00      0.00      0.00         2
      500112       1.00      0.50      0.67         2
      500116       0.22      1.00      0.36         4
      500117       1.00      0.20      0.33         5
      500118       0.00      0.00      0.00         3
      500119       0.73      0.89      0.80        18
      500152       0.00      0.00      0.00         0
      500153       1.00      0.33      0.50         3
      500155       1.00      1.00      1.00         1
      500156       1.00      0.67      0.80         3
      500229       0.67    

C:\Users\jastaron\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jastaron\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jastaron\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

In [34]:
cnt = 0
for i in range(len(train_str)):
    if train_PAC[i] != test_predict[i]:
        print(i,train_str[i],train_PAC[i],test_predict[i])
        cnt += 1
print(cnt,len(train_str),1-cnt/len(train_str))

0 四川凤凰山王家涝 348301 511922
1 四川万县地区中江石泉 500101 510623
2 四川省万县双和乡 500101 0
3 四川万县地区忠县乌羊区石子乡 500101 500233
4 四川万县地区巫山凤凰公社相柳池 500101 500237
30 四川江津地区大足双河区多宝公社建角大队 500111 500116
31 四川江津地区大足龙水玉龙公社五七中学 500111 500116
33 四川省峨眉县峨嵋山雷音寺 500112 511181
38 四川江津合川合川盐井华川厂附近 500117 500116
39 四川江津合川盐井华川厂附近 500117 500116
41 四川江津地区合川县盐井公社解放大队 500117 500116
42 四川江津地区合川盐井铁厂附近 500117 500116
43 四川江津地区永川石庙区九龙公社十大队 500118 500116
44 四川江津地区永川通用机械厂 500118 500116
45 四川江津（地区）永川石庙九龙公社一大队 500118 500116
48 四川涪陵地区南川三汇电厂沟 500119 500102
52 四川涪陵南川永安公社 500119 500102
64 四川江津地区荣昌盘龙区南药场 500153 500116
65 四川江津地区荣昌安北公社李家岩附近 500153 500116
69 四川涪陵地区武隆巷口凤山 500156 500102
113 四川涪陵地区石柱漆辽公社5大队 500240 500102
114 四川涪陵地区秀山城北门桥附近 500241 500102
115 四川涪陵地区凤凰山汽车站附近 500241 500102
116 四川涪陵地区秀山中和区沃土公社贵图大队河东 500241 500102
118 四川涪陵地区彭水城郊公社 500243 500102
119 四川涪陵地区彭水石盘公社 500243 500102
120 四川中国医学科学院西南中药研究所生物组 510105 0
121 四川省成都市成都中医药大学新区 510115 0
134 四川温江地区灌县 510181 510115
135 四川温江地区灌县青城山大师洞 510181 510115
140 四川温江地区彭县大宝公社5大队9队九道拐 510182 510115
158 四川渡口

In [18]:
temp1 = '四川凤凰山王家涝'
location = MatchCounty_new(temp1)
print(location)

['四川', '凤凰山', '王家']
county
hills
四川
凤凰山
王家
王家 王家寨
['四川省', '巴中市', '南江县', 511922]


In [72]:
hill_test_predict = []
hill_true_value = []
for i in hill_predict_index:
    hill_test_predict.append(test_predict[i])
    hill_true_value.append(train_PAC[i])

In [19]:
# 相似度匹配算法
'''
输入：
testList: 汉字列表，要计算和trainList的相似度
trainList：汉字列表，要计算和testList的相似度

'''
def SimilarityCal(testList,trainList,SimilarityScore):
    l = len(testList)
    SSCsum = 0
    for i in range(l):
        testStr = testList[i]
        if testStr != '':
            testIni = lpy(testStr,style=Style.INITIALS)[0]
            testIniCode = PyIniDict[testIni]
            testFinal = lpy(testStr,style=Style.NORMAL)[0].replace(testIni,'')
            testFinalCode = PyFinalDict[testFinal]
            testTone = lpy(testStr,style=Style.TONE3)[0][-1]
            TestPy = '%s%s%s' % (testIniCode,testFinalCode,testTone)
            TestWb = str(f'{wb(testStr)[0]:$<4}')
            TestBihua = strokesDict[get_stroke(testStr,strokes)]
            TestSSC = TestPy+TestWb+TestBihua
        else:
            TestSSC = '00000000'

        trainStr = trainList[i]
        if trainStr != '':
            trainIni = lpy(trainStr,style=Style.INITIALS)[0]
            trainIniCode = PyIniDict[trainIni]
            trainFinal = lpy(trainStr,style=Style.NORMAL)[0].replace(trainIni,'')
            trainFinalCode = PyFinalDict[trainFinal]
            trainTone = lpy(trainStr,style=Style.TONE3)[0][-1]
            TrainPy = '%s%s%s' % (trainIniCode,trainFinalCode,trainTone)
            TrainWb = str(f'{wb(trainStr)[0]:$<4}')
            TrainBihua = strokesDict[get_stroke(trainStr,strokes)]
            TrainSSC = TrainPy+TrainWb+TrainBihua
        else:
            TrainSSC = '00000000'

        # print(TestSSC+'\t'+TrainSSC)

        def CComp(char1,char2):
            if char1 == char2: return 1
            else: return 0

        te = list(TestSSC)
        tr = list(TrainSSC)
        SSCscore = 0
        for j in range(len(te)):
            if j in [3,4,5,6]:
                SSCscore += 0.15*CComp(te[j],tr[j])
            else:
                SSCscore += 0.1*CComp(te[j],tr[j])
            # SSCscore += 0.125*CComp(te[j],tr[j])
        SSCsum += SSCscore
    SSCsum /= l
    # SimilarityScore[''.join(trainList)] = SSCsum
    SimilarityScore.append([''.join(trainList),round(SSCsum,1)])

In [20]:
def countyAddrComp(p3):
    # countyPro = {}
    leastRatio = 0.4
    SimilarityScore = []
    TestLen = len(p3)
    for i in range(TestLen):
        for j in range(len(countyAbbr)):
            if p3[i] in countyAbbr[j]:
                trainAbbr = countyAbbr[j]
                LocaTrain = countyAbbr[j].find(p3[i])
                LocaTest = i
                # print(LocaTrain,LocaTest)
                TrainList = list(trainAbbr)
                TestList = list(p3)
                # print(TrainList,TestList)
                diff = LocaTrain - LocaTest
                while diff > 0:
                    TestList.insert(0,"")
                    diff -= 1
                while diff < 0:
                    TrainList.insert(0,"")
                    diff += 1
                lenDiff = len(TrainList)-len(TestList)
                while lenDiff > 0:
                    TestList.append("")
                    lenDiff -= 1
                while lenDiff < 0:
                    TrainList.append("")
                    lenDiff += 1
                SimilarityCal(TestList,TrainList,SimilarityScore)
    SimilarityScore.sort(key=(lambda x: x[1]),reverse=True)
    # print(SimilarityScore)
    if SimilarityScore[0] is not None and SimilarityScore[0][1] > leastRatio:
        return SimilarityScore[0]
    else:
        return None

In [21]:
# # data=str(input("请输入文本:"))
# temp = '四川省成都市双流区'
# # p3 = MatchCounty(temp)
# result = countyAddrComp()
# print(result)